In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [4]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


# No Line search

In [5]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()

opt = torch_numopt.GradientDescentLS(model=model, lr=0.005, line_search_method="const")

all_loss = {}
for epoch in range(1000):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].cpu().detach().numpy().item()))

epoch:  0, loss: 0.6761481165885925
epoch:  1, loss: 0.657284140586853
epoch:  2, loss: 0.6391590237617493
epoch:  3, loss: 0.6217473745346069
epoch:  4, loss: 0.6050043702125549
epoch:  5, loss: 0.5888789296150208
epoch:  6, loss: 0.5733609199523926
epoch:  7, loss: 0.5584023594856262
epoch:  8, loss: 0.5439767241477966
epoch:  9, loss: 0.5300385355949402
epoch:  10, loss: 0.5165624022483826
epoch:  11, loss: 0.5035180449485779
epoch:  12, loss: 0.49087804555892944
epoch:  13, loss: 0.4786328673362732
epoch:  14, loss: 0.46675780415534973
epoch:  15, loss: 0.455228328704834
epoch:  16, loss: 0.44402986764907837
epoch:  17, loss: 0.43314898014068604
epoch:  18, loss: 0.42257553339004517
epoch:  19, loss: 0.41229602694511414
epoch:  20, loss: 0.40230056643486023
epoch:  21, loss: 0.3925756812095642
epoch:  22, loss: 0.38311567902565
epoch:  23, loss: 0.3739073872566223
epoch:  24, loss: 0.3649451434612274
epoch:  25, loss: 0.3562214970588684
epoch:  26, loss: 0.34772858023643494
epoch: 

In [6]:
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = -190.5641404760785
Test metrics:  R2 = -193.2321866439647


# Backtrack line search

In [7]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()

opt = torch_numopt.GradientDescentLS(model=model, lr=1, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="armijo")
# opt = torch_numopt.GradientDescentLS(model=model, lr=1, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="wolfe")
# opt = torch_numopt.GradientDescentLS(model=model, lr=1, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="strong-wolfe")
# opt = torch_numopt.GradientDescentLS(model=model, lr=1, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="goldstein")

all_loss = {}
for epoch in range(1000):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].cpu().detach().numpy().item()))

epoch:  0, loss: 0.4261147975921631
epoch:  1, loss: 0.2413555085659027
epoch:  2, loss: 0.13654690980911255
epoch:  3, loss: 0.0811065286397934
epoch:  4, loss: 0.05408589541912079
epoch:  5, loss: 0.04195704311132431
epoch:  6, loss: 0.03688735142350197
epoch:  7, loss: 0.03487382456660271
epoch:  8, loss: 0.03409208357334137
epoch:  9, loss: 0.033784303814172745
epoch:  10, loss: 0.03364982455968857
epoch:  11, loss: 0.03357938677072525
epoch:  12, loss: 0.03342160955071449
epoch:  13, loss: 0.03326766565442085
epoch:  14, loss: 0.033196352422237396
epoch:  15, loss: 0.033091507852077484
epoch:  16, loss: 0.03292396292090416
epoch:  17, loss: 0.03284898027777672
epoch:  18, loss: 0.032813310623168945
epoch:  19, loss: 0.0326102189719677
epoch:  20, loss: 0.03252800554037094
epoch:  21, loss: 0.03248398378491402
epoch:  22, loss: 0.03230336681008339
epoch:  23, loss: 0.032213058322668076
epoch:  24, loss: 0.0321655198931694
epoch:  25, loss: 0.031997907906770706
epoch:  26, loss: 0.0

In [8]:
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = 0.7621390197729612
Test metrics:  R2 = 0.7524372927784362


## Bisection line search

In [9]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()

opt = torch_numopt.GradientDescentLS(model=model, lr=1, line_search_method="bisect")

all_loss = {}
for epoch in range(1000):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].cpu().detach().numpy().item()))

epoch:  0, loss: 0.16318592429161072
epoch:  1, loss: 0.048198092728853226
epoch:  2, loss: 0.03641186282038689
epoch:  3, loss: 0.035575322806835175
epoch:  4, loss: 0.03551589697599411
epoch:  5, loss: 0.03548041358590126
epoch:  6, loss: 0.03545035421848297
epoch:  7, loss: 0.03541966527700424
epoch:  8, loss: 0.03538878262042999
epoch:  9, loss: 0.035365670919418335
epoch:  10, loss: 0.035329774022102356
epoch:  11, loss: 0.035306416451931
epoch:  12, loss: 0.035271935164928436
epoch:  13, loss: 0.03524819388985634
epoch:  14, loss: 0.0352153517305851
epoch:  15, loss: 0.035190921276807785
epoch:  16, loss: 0.03515955060720444
epoch:  17, loss: 0.03513413667678833
epoch:  18, loss: 0.03510410711169243
epoch:  19, loss: 0.03507726266980171
epoch:  20, loss: 0.035048674792051315
epoch:  21, loss: 0.03502007946372032
epoch:  22, loss: 0.03499317541718483
epoch:  23, loss: 0.03496848791837692
epoch:  24, loss: 0.03493164852261543
epoch:  25, loss: 0.03490357846021652
epoch:  26, loss: 

In [10]:
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = 0.6895205934928381
Test metrics:  R2 = 0.6834542865074851
